---
Some useful $\LaTeX$ commands are defined in this cell:
$$
\newcommand{\abs}[1]{\left\lvert#1\right\rvert}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\paren}[1]{\left(#1\right)}
\newcommand{\brack}[1]{\left[#1\right]}
\newcommand{\ip}[2]{\left\langle#1,#2\right\rangle}
\DeclareMathOperator{\span}{span}
\DeclareMathOperator{\fl}{fl}
\abs{x}, \norm{x}, \set{x}, \paren{x}, \brack{x}, \ip{x}{y}, \span, \fl
$$

---

---
# 14.1 Deriving formulas using Taylor series
---

## A first-order method

In Chapter 1, we experimented with the approximation

$$ 
f'(x_0) \approx \frac{f(x_0 + h) - f(x_0)}{h}
$$

and saw that the **discretization error** (a.k.a. **truncation error**) satisfies

$$
\abs{f'(x_0) - \frac{f(x_0+h)-f(x_0)}{h}} \approx \frac{h}{2}\abs{ f''(x_0) } = \mathcal{O}(h)
$$

when $h$ is small enough and $f''(x_0) \neq 0$. We say that this approximation is **first-order accurate**.

---

For the numerical experiments in this section, we will use the function

$$f(x) = e^x \sin(3x).$$

In [1]:
using SymPy
t = symbols("t")
ft = exp(t)*sin(3t)
Sym[simplify(diff(ft, t, k)) for k=0:6]

7-element Vector{Sym}:
                      exp(t)*sin(3*t)
       (sin(3*t) + 3*cos(3*t))*exp(t)
    (-8*sin(3*t) + 6*cos(3*t))*exp(t)
  (-26*sin(3*t) - 18*cos(3*t))*exp(t)
   (28*sin(3*t) - 96*cos(3*t))*exp(t)
  (316*sin(3*t) - 12*cos(3*t))*exp(t)
 (352*sin(3*t) + 936*cos(3*t))*exp(t)

In [2]:
f(x)  = exp(x)*(                sin(3x))
f1(x) = exp(x)*(  3cos(3x) +    sin(3x))
f2(x) = exp(x)*(  6cos(3x) -   8sin(3x))
f3(x) = exp(x)*(-18cos(3x) -  26sin(3x))
f4(x) = exp(x)*(-96cos(3x) +  28sin(3x))
f5(x) = exp(x)*(-12cos(3x) + 316sin(3x))
f6(x) = exp(x)*(936cos(3x) + 352sin(3x))

f6 (generic function with 1 method)

In [3]:
using Printf

x0 = .4

@printf "%5s %14s %12s\n" "h" "approx" "error"
for i = 0:8
    h = 10.0^-i
    approx = (f(x0+h) - f(x0))/h
    err = abs(approx - f1(x0))
    @printf "%5.0e %14.10f %12.5e\n" h approx err
end

abs(f2(x0))/2

    h         approx        error
1e+00  -4.9248529710  7.93701e+00
1e-01   2.5415227422  4.70639e-01
1e-02   2.9719961407  4.01655e-02
1e-03   3.0082139897  3.94768e-03
1e-04   3.0117675904  3.94080e-04
1e-05   3.0121222691  3.94011e-05
1e-06   3.0121577301  3.94001e-06
1e-07   3.0121612760  3.94182e-07
1e-08   3.0121616312  3.89104e-08


3.940032967921215

---

## A second-order method

We also saw in Exercise 1.2 from HW1 that 

$$ f'(x_0) \approx \frac{f(x_0 + h) - f(x_0 - h)}{2h} $$

and that the truncation error satisfies

$$ \abs{f'(x_0) - \frac{f(x_0 + h) - f(x_0 - h)}{2h}} \approx \frac{h^2}{6}\abs{f'''(x_0)} = \mathcal{O}(h^2) $$

when $h$ is small enough and $f'''(x_0) \neq 0$. Thus, this approximation is **second-order accurate**.

In [4]:
x0 = .4

@printf "%5s %14s %12s\n" "h" "approx" "error"
for i = 0:8
    h = 10.0^-i
    approx = (f(x0+h) - f(x0-h))/(2h)
    err = abs(approx - f1(x0))
    @printf "%5.0e %14.10f %12.5e\n" h approx err
end

abs(f3(x0))/6

    h         approx        error
1e+00  -1.4999775659  4.51214e+00
1e-01   2.9360523683  7.61093e-02
1e-02   3.0113970104  7.64660e-04
1e-03   3.0121540232  7.64695e-06
1e-04   3.0121615937  7.64704e-08
1e-05   3.0121616694  7.63182e-10
1e-06   3.0121616701  5.26392e-11
1e-07   3.0121616701  5.26392e-11
1e-08   3.0121616645  5.60375e-09


7.646958095519647

---

## Overview

In this section, we will use the **Taylor series** of $f(x)$ derive these and other formulas to approximate the first and second derivatives of a function $f(x)$.

These approximations will use the value of $f(x)$ at evenly-spaced points:

$$\ldots, \ x_0 - 2h, \quad x_0 - h, \quad x_0, \quad x_0 + h, \quad x_0 + 2h, \ \ldots$$



## Recall:

> ## Theorem: (Taylor Series)
>
> If $f$ is a function that is $(k+1)$-differentiable on an interval containing $x_0$ and $x_0 + h$, then
>
> $$
f(x_0 + h) = f(x_0) + h f'(x_0) + \frac{h^2}{2} f''(x_0) + \cdots + \frac{h^k}{k!} f^{(k)}(x_0) + \frac{h^{k+1}}{(k+1)!} f^{(k+1)}(\xi),
$$
>
> for some $\xi \in (x_0, x_0 + h)$.

---

## Two-point formulas

---

### Forward difference approximation for $f'(x_0)$

Let $f$ be twice-differentiable on an interval containing $x_0$ and $x_0 + h$. From the Taylor series, we have

$$f(x_0 + h) = f(x_0) + hf'(x_0) + \frac{h^2}{2}f''(\xi), \quad \text{for some $\xi \in (x_0,x_0+h).$}$$

Solving for $f'(x_0)$, we obtain

$$f'(x_0) = \frac{f(x_0+h) - f(x_0)}{h} - \frac{h}{2}f''(\xi).$$

Thus, the **forward** difference formula 

$$\fbox{${\displaystyle\frac{f(x_0+h) - f(x_0)}{h}}$}$$

gives a first order (i.e., $\mathcal{O}(h)$) approximation of $f'(x_0)$:

$$\abs{f'(x_0) - \frac{f(x_0+h) - f(x_0)}{h}} = \frac{h}{2}\abs{f''(\xi)}, \quad \text{for some $\xi \in (x_0,x_0+h).$}$$

### Exercise: Backward difference approximation for $f'(x_0)$

Let $f$ be twice-differentiable on an interval containing $x_0$ and $x_0-h$. From the Taylor series, we have

$$f(x_0 - h) = f(x_0) - hf'(x_0) + \frac{h^2}{2}f''(\xi), \quad \text{for some $\xi \in (x_0-h,x_0).$}$$

Use this Taylor series to obtain the **backward** difference approximation of $f'(x_0)$. Show that this approximation is also first order.

Solving for $f'(x_0)$, we obtain

$$f'(x_0) = \frac{f(x_0) - f(x_0-h)}{h} + \frac{h}{2}f''(\xi).$$

Thus, the **backward** difference formula 

$$\fbox{${\displaystyle\frac{f(x_0) - f(x_0-h)}{h}}$}$$

gives a first order (i.e., $\mathcal{O}(h)$) approximation of $f'(x_0)$:

$$\abs{f'(x_0) - \frac{f(x_0) - f(x_0-h)}{h}} = \frac{h}{2}\abs{f''(\xi)}, \quad \text{for some $\xi \in (x_0-h,x_0).$}$$

In [5]:
@printf "%5s %14s %12s\n" "h" "approx" "error"
for i = 0:8
    h = 10.0^-i
    approx = (f(x0) - f(x0-h))/h
    err = abs(approx - f1(x0))
    @printf "%5.0e %14.10f %12.5e\n" h approx err
end

abs(f2(x0))/2

    h         approx        error
1e+00   1.9248978392  1.08726e+00
1e-01   3.3305819944  3.18420e-01
1e-02   3.0507978801  3.86362e-02
1e-03   3.0160940567  3.93239e-03
1e-04   3.0125555970  3.93927e-04
1e-05   3.0122010697  3.93996e-05
1e-06   3.0121656101  3.93991e-06
1e-07   3.0121620642  3.94077e-07
1e-08   3.0121616978  2.77029e-08


3.940032967921215

---

## Three-point formulas

---

### Centered difference approximation for $f'(x_0)$

Let $f$ be thrice-*continuously*-differentiable on an interval containing $x_0-h$ and $x_0+h$. From the Taylor series, we have

$$
\begin{align}
f(x_0 + h) &= f(x_0) + hf'(x_0) + \frac{h^2}{2}f''(x_0) + \frac{h^3}{6}f'''(\xi_1), \quad \text{for some $\xi_1 \in (x_0,x_0+h),$} \\
f(x_0 - h) &= f(x_0) - hf'(x_0) + \frac{h^2}{2}f''(x_0) - \frac{h^3}{6}f'''(\xi_2), \quad \text{for some $\xi_2 \in (x_0-h,x_0).$}\\
\end{align}
$$

Subtracting the second from the first, we get

$$f(x_0 + h) - f(x_0 - h) = 2hf'(x_0) + \frac{h^3}{6}\paren{f'''(\xi_1) + f'''(\xi_2)}.$$

Since $f'''$ is continous, the Intermediate Value Theorem tells us that there exists $\xi$ strictly between $\xi_1$ and $\xi_2$ such that

$$f'''(\xi) = \frac12\paren{f'''(\xi_1) + f'''(\xi_2)}.$$

Therefore, we have

$$f(x_0 + h) - f(x_0 - h) = 2hf'(x_0) + \frac{h^3}{3}f'''(\xi),$$

and solving for $f'(x_0)$ gives us

$$f'(x_0) = \frac{f(x_0 + h) - f(x_0 - h)}{2h} - \frac{h^2}{6}f'''(\xi).$$

Thus, the **centered** difference formula

$$\fbox{${\displaystyle \frac{f(x_0 + h) - f(x_0 - h)}{2h}}$}$$

gives a second order (i.e., $\mathcal{O}(h^2)$) approximation of $f'(x_0)$:

$$\abs{f'(x_0) -  \frac{f(x_0 + h) - f(x_0 - h)}{2h}} = \frac{h^2}{6}\abs{f'''(\xi)}, \quad \text{for some $\xi \in (x_0-h,x_0+h).$}$$

---

### One-sided three-point approximation for $f'(x_0)$

Let $f$ be four-times differentiable on an interval containing $x_0$ and $x_0 + 2h$. From the Taylor series, we have

$$
\begin{align}
f(x_0 + h) &= f(x_0) + hf'(x_0) + \frac{h^2}{2}f''(x_0) + \frac{h^3}{6}f'''(x_0) + \mathcal{O}(h^4), \\
f(x_0 + 2h) &= f(x_0) + 2hf'(x_0) + 2h^2f''(x_0) + \frac{4h^3}{3}f'''(x_0) + \mathcal{O}(h^4). \\
\end{align}
$$

Let's take $y_1$ times the first equation *plus* $y_2$ times the second equation:

$$
y_1 f(x_0 + h) + y_2 f(x_0 + 2h) = \\ 
(y_1 + y_2) f(x_0)  \\
+ (y_1 + 2y_2)hf'(x_0) \\
+\paren{\frac{1}{2}y_1 + 2y_2} h^2f''(x_0) \\
+\paren{\frac{1}{6}y_1 + \frac{4}{3} y_2} h^3f'''(x_0) +\mathcal{O}(h^4).
$$

To keep the $f'(x_0)$ term and delete the $f''(x_0)$ term, we choose $y_1$ and $y_2$ such that

$$
\begin{align}
y_1 + 2 y_2 &= 1, \\
\frac{1}{2} y_1 + 2y_2 &= 0.\\
\end{align}
$$

Thus, we just need to solve the linear system

$$
\begin{bmatrix}
1 & 2 \\
\frac12 & 2 \\
\end{bmatrix}
\begin{bmatrix}
y_1 \\
y_2 \\
\end{bmatrix}
=
\begin{bmatrix}
1 \\
0 \\
\end{bmatrix}.
$$

In [ ]:
A = [
    1     2
    1//2  2
]
b = [1, 0]

y1, y2 = A\b

In [ ]:
y1 + y2

In [ ]:
y1/6 + 4//3*y2

Therefore, $y_1 = 2$ and $y_2 = -\frac12$. Thus,

$$
2 f(x_0 + h) -\frac12 f(x_0 + 2h) 
= \frac32 f(x_0) + hf'(x_0) -\frac{h^3}{3}f'''(x_0) + \mathcal{O}(h^4).
$$

Then multiplying both sides by 2, we get

$$
4 f(x_0 + h) - f(x_0 + 2h) 
= 3 f(x_0) + 2hf'(x_0) -\frac{2h^3}{3}f'''(x_0) + \mathcal{O}(h^4).
$$



Solving for $f'(x_0)$, we obtain

$$ f'(x_0) =  \frac{- 3f(x_0) + 4f(x_0 + h) - f(x_0 + 2h)}{2h} + \frac{h^2}{3} f'''(x_0) + \mathcal{O}(h^3). $$

Thus, the **one-sided three-point** difference formula

$$\fbox{${\displaystyle \frac{- 3f(x_0) + 4f(x_0 + h) - f(x_0 + 2h)}{2h}}$}$$

gives a second order (i.e., $\mathcal{O}(h^2)$) approximation of $f'(x_0)$:

$$\abs{f'(x_0) -  \frac{- 3f(x_0) + 4f(x_0 + h) - f(x_0 + 2h)}{2h}} \leq \frac{h^2}{3}\abs{f'''(x_0)} + \mathcal{O}(h^3).$$

In [ ]:
x0 = .4

@printf "%5s %14s %12s\n" "h" "approx" "error"
for i = 0:10
    h = 10.0^-i
    approx = (-3f(x0) + 4f(x0 + h) - f(x0 + 2h))/(2h)
    err = abs(approx - f1(x0))
    @printf "%5.0e %14.10f %12.5e\n" h approx err
end

abs(f3(x0))/3

### Note:

It can be shown using a more complicated argument that if $f$ is thrice-continuously-differentiable on an interval containing $x_0$ and $x_0 + 2h$, then

$$\abs{f'(x_0) -  \frac{- 3f(x_0) + 4f(x_0 + h) - f(x_0 + 2h)}{2h}} = \frac{h^2}{3}\abs{f'''(\xi)}, \quad \text{for some $\xi \in (x_0,x_0+2h).$}$$

---

### Centered three-point approximation for $f''(x_0)$

Let $f$ be four-times continuously-differentiable on an interval containing $x_0 - h$ and $x_0 + h$. From the Taylor series, we have

$$
\begin{align}
f(x_0 + h) &= f(x_0) + hf'(x_0) + \frac{h^2}{2}f''(x_0) + \frac{h^3}{6}f'''(x_0) + \frac{h^4}{24} f^{(iv)}(\xi_1), \quad \text{for some $\xi_1 \in (x_0,x_0+h),$} \\
f(x_0 - h) &= f(x_0) - hf'(x_0) + \frac{h^2}{2}f''(x_0) - \frac{h^3}{6}f'''(x_0) + \frac{h^4}{24} f^{(iv)}(\xi_2), \quad \text{for some $\xi_2 \in (x_0-h,x_0).$}\\
\end{align}
$$

This time we add the Taylor expansions to obtain

$$
f(x_0 + h) + f(x_0 - h) = 2f(x_0) + h^2f''(x_0) + \frac{h^4}{24} \left( f^{(iv)}(\xi_1) + f^{(iv)}(\xi_2) \right)
$$

Since $f^{(iv)}$ is continous, the Intermediate Value Theorem tells us that there exists $\xi$ strictly between $\xi_1$ and $\xi_2$ such that

$$f^{(iv)}(\xi) = \frac12\paren{f^{(iv)}(\xi_1) + f^{(iv)}(\xi_2)}.$$

Therefore, we have

$$
f(x_0 + h) + f(x_0 - h) = 2f(x_0) + h^2f''(x_0) + \frac{h^4}{12} f^{(iv)}(\xi),
$$

and solving for $f''(x_0)$ gives us

$$f''(x_0) = \frac{f(x_0 - h) - 2f(x_0) + f(x_0 + h)}{h^2} - \frac{h^2}{12}f^{(iv)}(\xi).$$

Thus, the **centered** difference formula

$$\fbox{${\displaystyle \frac{f(x_0 - h) - 2f(x_0) + f(x_0 + h)}{h^2}}$}$$

gives a second order (i.e., $\mathcal{O}(h^2)$) approximation of $f''(x_0)$:

$$\abs{f''(x_0) -  \frac{f(x_0 - h) - 2f(x_0) + f(x_0 + h)}{h^2}} = \frac{h^2}{12}\abs{f^{(iv)}(\xi)}, \quad \text{for some $\xi \in (x_0-h,x_0+h).$}$$

In [ ]:
x0 = .4

@printf "%5s %14s %12s\n" "h" "approx" "error"
for i = 0:8
    h = 10.0^-i
    approx = (f(x0 - h) - 2f(x0) + f(x0 + h))/h^2
    err = abs(approx - f2(x0))
    @printf "%5.0e %14.10f %12.5e\n" h approx err
end

abs(f4(x0))/12

---

## Five-point formulas

---

### Centered five-point approximation for $f'(x_0)$

Let $f$ be seven-times differentiable on an interval containing $x_0-2h$ and $x_0 + 2h$. From the Taylor series, we have

$$
\begin{align}
f(x_0 + h) &= k_0 + hk_1 + h^2k_2 + h^3k_3 + h^4k_4 + h^5k_5 + h^6k_6 + \mathcal{O}(h^7), \\ 
f(x_0 - h) &= k_0 - hk_1 + h^2k_2 - h^3k_3 + h^4k_4 - h^5k_5 + h^6k_6 + \mathcal{O}(h^7), \\
f(x_0 + 2h) &= k_0 + 2hk_1 + 4h^2k_2 + 8h^3k_3 + 16h^4k_4 + 32h^5k_5 + 64h^6k_6 + \mathcal{O}(h^7), \\ 
f(x_0 - 2h) &= k_0 - 2hk_1 + 4h^2k_2 - 8h^3k_3 + 16h^4k_4 - 32h^5k_5 + 64h^6k_6 + \mathcal{O}(h^7), \\
\end{align}
$$

where $k_n = \frac{f^{(n)}(x_0)}{n!}$, for $n = 0,\ldots,6$.

We multiple the above equations respectively by $y_1, y_2, y_3, y_4$ and add them to obtain

$$
y_1f(x_0 + h) + y_2f(x_0 - h) + y_3f(x_0 - 2h) + y_4f(x_0 + 2h) = \\
(y_1 + y_2 + y_3 + y_4)k_0 \\
+ (y_1 - y_2 + 2y_3 - 2y_4)hk_1 \\
+ (y_1 + y_2 + 4y_3 + 4y_4)h^2k_2 \\
+ (y_1 - y_2 + 8y_3 - 8y_4)h^3k_3 \\
+ (y_1 + y_2 + 16y_3 + 16y_4)h^4k_4 \\
+ (y_1 - y_2 + 32y_3 - 32y_4)h^5k_5 \\
+ (y_1 + y_2 + 64y_3 + 64y_4)h^6k_6 + \mathcal{O}(h^7).
$$

We can specify four conditions to solve for $y_1, y_2, y_3, y_4$. We want to keep the $f'(x_0)$ term, and get the highest accuracy possible. Thus, we want

$$
\begin{align}
y_1 - y_2 + 2 y_3 - 2y_4 &= 1,\\
y_1 + y_2 + 4y_3 + 4y_4 &= 0,\\
y_1 - y_2 + 8 y_3 - 8y_4 &= 0,\\
y_1 + y_2 + 16y_3 + 16y_4 &= 0.\\
\end{align}
$$

In [ ]:
y = [1 -1 2 -2; 1 1 4 4; 1 -1 8 -8; 1 1 16 16//1]\[1,0,0,0]

In [ ]:
y *= 12

In [ ]:
y[1] + y[2] + y[3] + y[4]

In [ ]:
y[1] - y[2] + 32y[3] -32y[4]

In [ ]:
y[1] + y[2] + 64y[3] + 64y[4]

Thus, we have

$$
8f(x_0 + h) - 8f(x_0 - h) - f(x_0 - 2h) + f(x_0 + 2h) = \\
12hf'(x_0) - 48h^5\frac{f^{(v)}(x_0)}{120} + \mathcal{O}(h^7).
$$

Solving for $f'(x_0)$, we obtain:

$$
f'(x_0) =  \frac{f(x_0 - 2h) - 8f(x_0 - h)  + 8f(x_0 + h) - f(x_0 + 2h)}{12h} \\
+ \frac{h^4}{30} f^{(v)}(x_0) + \mathcal{O}(h^6)
$$

Thus, the **centered five-point** difference formula

$$\fbox{${\displaystyle \frac{f(x_0 - 2h) - 8f(x_0 - h)  + 8f(x_0 + h) - f(x_0 + 2h)}{12h}}$}$$

gives a fourth order (i.e., $\mathcal{O}(h^4)$) approximation of $f'(x_0)$:

$$\abs{f'(x_0) -  \frac{f(x_0 - 2h) - 8f(x_0 - h)  + 8f(x_0 + h) - f(x_0 + 2h)}{12h}} \\
\leq \frac{h^4}{30}\abs{f^{(v)}(x_0)} + \mathcal{O}(h^6).$$

In [ ]:
#x0 = 0.4
x0 = big".4"

@printf "%5s %14s %12s\n" "h" "approx" "error"
for i = 0:8
#    h = 10.0^-i
    h = BigFloat("10.0")^-i
    approx = (f(x0 - 2h) - 8f(x0 - h) + 8f(x0 + h) - f(x0 + 2h))/(12h)
    err = abs(approx - f1(x0))
    @printf "%5.0e %14.10f %12.5e\n" h approx err
end

abs(f5(x0))/30

### Note:

It can be shown using a more complicated argument that if $f$ is five-times-continuously-differentiable on an interval containing $x_0-2h$ and $x_0 + 2h$, then

$$\abs{f'(x_0) -  \frac{f(x_0 - 2h) - 8f(x_0 - h)  + 8f(x_0 + h) - f(x_0 + 2h)}{12h}} = \frac{h^4}{30}\abs{f^{(v)}(\xi)}, \quad \text{for some $\xi \in (x_0-2h,x_0+2h).$}$$

---

### Exercise:

Derive a difference formula for the **fourth** derivative of $f$ at $x_0$ using Taylor's expansions at $x_0 \pm h$ and $x_0 \pm 2h$. How many points will be used in total and what is the expected order of the resulting formula?

Recall that

$$
\begin{align}
f(x_0 + h) &= k_0 + hk_1 + h^2k_2 + h^3k_3 + h^4k_4 + h^5k_5 + h^6k_6 + h^7k_7 + \mathcal{O}(h^8), \\ 
f(x_0 - h) &= k_0 - hk_1 + h^2k_2 - h^3k_3 + h^4k_4 - h^5k_5 + h^6k_6 - h^7k_7 + \mathcal{O}(h^8), \\
f(x_0 + 2h) &= k_0 + 2hk_1 + 4h^2k_2 + 8h^3k_3 + 16h^4k_4 + 32h^5k_5 + 64h^6k_6 + 128h^7k_7 + \mathcal{O}(h^8), \\ 
f(x_0 - 2h) &= k_0 - 2hk_1 + 4h^2k_2 - 8h^3k_3 + 16h^4k_4 - 32h^5k_5 + 64h^6k_6 - 128h^7k_7 + \mathcal{O}(h^8), \\
\end{align}
$$

where $k_n = \frac{f^{(n)}(x_0)}{n!}$, for $n = 0,\ldots,6$.

---